En este notebook se van a comparar las metricas de precisión de los modelos generados anteriormente.

## Importar librerias

In [1]:
import os
from pathlib import PurePath
import json
import pandas as pd

## Busqueda de los ficheros de metricas

In [2]:
root = r"C:\Users\Isai\Documents\Tesis\code\data_analysis\linear_reg\coeficientes\zafra2022\ndvi_mean"

In [3]:
files = [(json.load(open(PurePath(root, file), "r")), int(file.split('.')[0].split('_')[1])) for file in os.listdir(root) if file.endswith(".json")]

In [4]:
raw_data = {
    'parcela': [],
    'mse': [],
    'aic': [],
    'rsquared': []
}

for i in files:
    raw_data['parcela'].append(i[1])
    raw_data['mse'].append(i[0][1]['mse'])
    raw_data['aic'].append(i[0][1]['aic'])
    raw_data['rsquared'].append(i[0][1]['rsquared'])

df = pd.DataFrame(raw_data)
    

In [6]:
df.describe()

,parcela,mse,aic,rsquared
count,15.000000,15.000000,15.000000,15.000000
mean,8.266667,0.488976,-80.239085,0.923261
std,4.832430,0.134259,19.189412,0.050621
min,1.000000,0.252783,-118.682431,0.817460
25%,4.500000,0.401106,-95.439949,0.898419
50%,8.000000,0.496827,-78.994767,0.930419
75%,12.000000,0.570783,-65.250206,0.962592
max,16.000000,0.737323,-55.981649,0.985646
